# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("./output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santiago de cao,-7.9589,-79.2392,21.07,83,40,3.60,PE,1691032427
1,1,kavaratti,10.5669,72.6420,28.26,78,100,6.58,IN,1691032249
2,2,yellowknife,62.4560,-114.3525,15.29,72,52,3.11,CA,1691032293
3,3,tralee,52.2704,-9.7026,13.75,92,100,6.88,IE,1691032428
4,4,howrah,22.5892,88.3103,29.00,84,75,3.09,IN,1691032428


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    # scale = 0.5,
    color = "City"
)
# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [7]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     574 non-null    int64  
 1   City        574 non-null    object 
 2   Lat         574 non-null    float64
 3   Lng         574 non-null    float64
 4   Max Temp    574 non-null    float64
 5   Humidity    574 non-null    int64  
 6   Cloudiness  574 non-null    int64  
 7   Wind Speed  574 non-null    float64
 8   Country     571 non-null    object 
 9   Date        574 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 45.0+ KB


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

city_df=city_data_df[
    (city_data_df['Max Temp']>21) & 
    (city_data_df['Max Temp']<27) &
    (city_data_df['Wind Speed']<4.5) &
    (city_data_df['Cloudiness']<20)
                ]

# Drop any rows with null values
city_df=city_df.dropna()

# Display sample data
city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,afaahiti,-17.7500,-149.2833,25.93,65,5,4.17,PF,1691032163
13,13,hamilton,39.1834,-84.5333,24.32,76,0,1.54,US,1691032354
21,21,rignano flaminio,42.2063,12.4794,23.22,90,0,0.22,IT,1691032429
80,80,izhmorskiy,56.1940,86.6414,23.98,61,11,2.89,RU,1691032416
81,81,carutapera,-1.1950,-46.0200,24.43,92,5,2.55,BR,1691032185


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df=city_df.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name']=''
# Display sample data

hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
7,afaahiti,PF,-17.7500,-149.2833,65,
13,hamilton,US,39.1834,-84.5333,76,
21,rignano flaminio,IT,42.2063,12.4794,90,
80,izhmorskiy,RU,56.1940,86.6414,61,
81,carutapera,BR,-1.1950,-46.0200,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    latitude=row['Lat']
    longitude=row['Lng']
    

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
afaahiti - nearest hotel: Omati Lodge
hamilton - nearest hotel: North Vista Manor
rignano flaminio - nearest hotel: Hotel Gran Sasso
izhmorskiy - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
paphos - nearest hotel: Mayfair Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
tsarevo - nearest hotel: Хотел "Диана"
humberto de campos - nearest hotel: No hotel found
papao - nearest hotel: Hiti Moana Villa Lodge
suez - nearest hotel: فندق لوسيندا
qarazhal - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
nar'yan-mar - nearest hotel: Гостиница "Агат"
kokomo - nearest hotel: Comfort Inn
bairnsdale - nearest hotel: Main hotel
griffith - nearest hotel: Quality Inn & Suites
tobarra - nearest hotel: Hotel Reina Victoria
greenville - nearest hotel: Quality Inn
remire-montjoly - nearest hotel: Complexe Belova
'ain el melh - nearest hotel: No hotel found
morehead city - nearest hotel: Bask Hotel Big Rock L

,City,Country,Lat,Lng,Humidity,Hotel Name
7,afaahiti,PF,-17.7500,-149.2833,65,Omati Lodge
13,hamilton,US,39.1834,-84.5333,76,North Vista Manor
21,rignano flaminio,IT,42.2063,12.4794,90,Hotel Gran Sasso
80,izhmorskiy,RU,56.1940,86.6414,61,No hotel found
81,carutapera,BR,-1.1950,-46.0200,92,No hotel found
85,paphos,CY,34.7667,32.4167,56,Mayfair Hotel
124,fort bragg,US,35.1390,-79.0060,70,Airborne Inn Lodging
126,tsarevo,BG,42.1667,27.8500,66,"Хотел ""Диана"""
149,humberto de campos,BR,-2.5983,-43.4611,92,No hotel found
150,papao,PF,-17.7500,-149.5500,62,Hiti Moana Villa Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    # size = "Distance",
    # scale = 0.5,
    color = "City",
    hover_cols=['Humidity','Hotel Name','Country']

)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)